<a href="https://colab.research.google.com/github/dvschultz/ShapeMatchingGAN/blob/master/ShapeMatchingGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Shape Matching GAN

Shape Matching GAN converts imaage textures to fit in shapes. Is it like style transfer? Kind of but no. Is it like CycleGAN? Also kind of, but no.

See [the project page](https://williamyang1991.github.io/projects/ICCV2019/SMGAN.html) for more.

I’ll show you how to load the library, run examples using the pre-trained models, and then train your own model. Be aware training can take up to the limit of Colab so you might want to save it to Google Drive just to be safe.

# Install Library and required data

In [0]:
!git clone https://github.com/dvschultz/ShapeMatchingGAN
%cd ShapeMatchingGAN/save
!gdown --id 1gjHR39deUSPChtRbKAD80waoQFTiXyMs #download example checkpoint files
!unzip ShapeMGAN-data_plus_model.zip
!rm -r data/style
!mkdir ../data/rawtext/yaheiB/train
!mv data/rawtext/yaheiB/train/*.* ../data/rawtext/yaheiB/train/
!mkdir ../data/rawtext/augment
!mv data/rawtext/augment/*.* ../data/rawtext/augment/
!mv save/*.* ./
!rmdir save
!rm ShapeMGAN-data_plus_model.zip
!rm -r data
%cd ../src

Cloning into 'ShapeMatchingGAN'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 633 (delta 18), reused 12 (delta 2), pack-reused 592
Receiving objects: 100% (633/633), 7.33 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (299/299), done.
/content/ShapeMatchingGAN/save
Downloading...
From: https://drive.google.com/uc?id=1gjHR39deUSPChtRbKAD80waoQFTiXyMs
To: /content/ShapeMatchingGAN/save/ShapeMGAN-data_plus_model.zip
118MB [00:04, 28.0MB/s]
Archive:  ShapeMGAN-data_plus_model.zip
   creating: data/
   creating: data/rawtext/
   creating: data/rawtext/augment/
  inflating: data/rawtext/augment/0000.png  
  inflating: data/rawtext/augment/0001.png  
  inflating: data/rawtext/augment/0002.png  
  inflating: data/rawtext/augment/0003.png  
  inflating: data/rawtext/augment/0004.png  
   creating: data/rawtext/yaheiB/
   creating: data/rawtext/yaheiB/train/
  inflating: data/rawtext/yaheiB/

Optionally, if you want to do training I recommend connecting a Google Drive account to Colab.

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


# Test
Let’s run a sample test to see how it does. This will generate a file named 'output.png' in your `/output/` folder.

There are four pretrained models to train from: fire, maple leaves, water, and smoke.

In [0]:
!python test.py --scale 0.0 --structure_model ../save/fire-GS-iccv.ckpt --texture_model ../save/fire-GT-iccv.ckpt --gpu

--- load options ---
gpu: True
name: output
result_dir: ../output/
scale: 0.0
scale_step: 0.2
structure_model: ../save/fire-GS-iccv.ckpt
text_name: ../data/rawtext/yaheiB/val/0801.png
text_type: 0
texture_model: ../save/fire-GT-iccv.ckpt
--- load data ---
--- load model ---
--- testing ---
--- save ---


Another example with some additional arguments. This will generate multiple images at different “scales.” It will produce something like an animation when all the frames are combined.

In [0]:
!python test.py --text_name ../data/rawtext/yaheiB/val/0801.png --scale -1 --scale_step 0.005 --structure_model ../save/fire-GS-iccv.ckpt --texture_model ../save/fire-GT-iccv.ckpt --result_dir ../output --name fire-0801 --gpu

--- load options ---
gpu: True
name: ladiescrop-0801
result_dir: ../output
scale: -1.0
scale_step: 0.005
structure_model: /content/ShapeMatchingGAN/save/ladiescrop-GS.ckpt
text_name: ../data/rawtext/yaheiB/val/0801.png
text_type: 0
texture_model: /content/ShapeMatchingGAN/save/ladiescrop-GT.ckpt
--- load data ---
--- load model ---
--- testing ---
--- save ---


We can zip all of these images up and download them. Once finished, in the Files tab on the left you’ll see a file named `fire_images.zip` Control-click on it and select `Download`.

In [0]:
!zip /content/fire_images.zip /content/ShapeMatchingGAN/output/fire-0801_*.png

  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_0.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_100.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_101.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_102.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_103.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_104.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_105.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_106.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_107.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_108.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_109.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/ladiescrop-0801_10.png (deflated 0%)
  adding: content/ShapeMatchingGAN/output/l

# Train a custom model
Training a custom model happens in 3 different training parts. From what I can tell you don’t need to train the Sketch Module—the pretrained model is fine.

## Image Pre-processing
To train a custom model you need to process your image in the format SMGAN requires.

First, upload two images to Colab. The first image should be the image you want the texture/style from. The second image should be a masked (black/white) version of this image. See the examples in the `uploads` folder.

Once uploaded, run the next command with the paths for each image as arguments.

In [0]:
!python preprocess.py --image ../uploads/sample_v3.png --image_mask ../uploads/sample_mask_v3.png

When that cell has finished, there will be a new image in your `data/style` folder. Use this file with the `--style_name` argument when training.

## Train the Sketch Module (optional)
The Sketch Module is trained on the letter shapes. Unless you have a specific need you can use the pre-trained model for this.

In [0]:
# !python trainSketchModule.py --help
!python trainSketchModule.py --text_path ../data/rawtext/yaheiB/train --text_datasize 708 --augment_text_path ../data/rawtext/augment --augment_text_datasize 5 --batchsize 16 --Btraining_num 12800 --save_GB_name ../save/GB.ckpt --gpu

--- load options ---
Btraining_num: 12800
DB_nf: 32
DB_nlayers: 5
GB_nf: 32
GB_nlayers: 6
augment_text_datasize: 5
augment_text_path: ../data/rawtext/augment
batchsize: 16
epochs: 3
gpu: True
save_GB_name: ../save/GB.ckpt
text_datasize: 708
text_path: ../data/rawtext/yaheiB/train
--- create model ---
--- training ---
Epoch [1/3][001/800]: LDadv: +0.017, LGadv: +0.152, Lrec: +81.663
Epoch [1/3][002/800]: LDadv: +0.061, LGadv: +0.158, Lrec: +63.030
Epoch [1/3][003/800]: LDadv: +0.086, LGadv: +0.158, Lrec: +49.606
Epoch [1/3][004/800]: LDadv: +0.069, LGadv: +0.152, Lrec: +36.851
Epoch [1/3][005/800]: LDadv: +0.078, LGadv: +0.175, Lrec: +26.935
Epoch [1/3][006/800]: LDadv: +0.057, LGadv: +0.174, Lrec: +22.889
Epoch [1/3][007/800]: LDadv: +0.049, LGadv: +0.147, Lrec: +18.177
Epoch [1/3][008/800]: LDadv: +0.037, LGadv: +0.146, Lrec: +17.177
Epoch [1/3][009/800]: LDadv: +0.022, LGadv: +0.173, Lrec: +15.270
Epoch [1/3][010/800]: LDadv: +0.025, LGadv: +0.158, Lrec: +13.684
Epoch [1/3][011/800]:

## Train the Structure Transfer model

The Structure Transfer model will take the most time to train—usually it takes a few hours to train. 

When completed there will be two new models in the `/save` folder. One is labeled {name}-GS.ckpt and the other {name}-DS.ckpt. You should download the GS as soon as its done—you don’t need the DS checkpoint unless you want to resume training at a later date.

In [0]:
!python trainStructureTransfer.py --style_name ../data/style/sample_v3.png --batchsize 16 --Straining_num 2560 --step1_epochs 30 --step2_epochs 40 --step3_epochs 80 --scale_num 4 --Sanglejitter --save_path ../save --save_name sample_v3 --gpu
#!python trainStructureTransfer.py --style_name ../data/style/ladiescrop-720.png --batchsize 16 --Straining_num 2560 --step1_epochs 30 --step2_epochs 40 --step3_epochs 80 --scale_num 4 --Sanglejitter --save_path ../save --save_name ladies --gpu

Streaming output truncated to the last 5000 lines.
Step3, Epoch [49/80][123/160]: LDadv: +0.553, LGadv: +13.346, Lrec: +4.591, Lgly: +0.000
Step3, Epoch [49/80][124/160]: LDadv: +0.714, LGadv: +12.889, Lrec: +5.253, Lgly: +0.000
Step3, Epoch [49/80][125/160]: LDadv: +0.574, LGadv: +13.094, Lrec: +3.893, Lgly: +0.000
Step3, Epoch [49/80][126/160]: LDadv: +0.834, LGadv: +12.794, Lrec: +5.740, Lgly: +0.000
Step3, Epoch [49/80][127/160]: LDadv: +0.597, LGadv: +13.137, Lrec: +5.038, Lgly: +0.000
Step3, Epoch [49/80][128/160]: LDadv: +0.521, LGadv: +13.422, Lrec: +4.228, Lgly: +0.000
Step3, Epoch [49/80][129/160]: LDadv: +0.492, LGadv: +13.304, Lrec: +4.528, Lgly: +0.000
Step3, Epoch [49/80][130/160]: LDadv: +0.434, LGadv: +13.351, Lrec: +4.786, Lgly: +0.000
Step3, Epoch [49/80][131/160]: LDadv: +0.617, LGadv: +12.985, Lrec: +5.817, Lgly: +0.000
Step3, Epoch [49/80][132/160]: LDadv: +0.644, LGadv: +12.881, Lrec: +5.482, Lgly: +0.000
Step3, Epoch [49/80][133/160]: LDadv: +0.582, LGadv: +13.06

## Train the Texture Transfer
The Texture Transfer model will take the some time to train, but is much faster than the Structure model. It usually takes me an ~hour or so. 

When completed there will be two new models in the `/save` folder. One is labeled {name}-GT.ckpt and the other {name}-DT.ckpt. You should download the GT as soon as its done—you don’t need the DT checkpoint unless you want to resume training at a later date.

In [0]:
!python trainTextureTransfer.py --style_name ../data/style/sample_v3.png --batchsize 4 --Ttraining_num 800 --texture_step1_epochs 40 --save_path ../save --save_name sample_v3-styleloss --gpu --style_loss --load_GS_name ../save/sample_v3-GS.ckpt

--- load options ---
DB_nf: 32
DB_nlayers: 5
DS_nf: 32
DS_nlayers: 4
DT_nf: 32
DT_nlayers: 4
GB_nf: 32
GB_nlayers: 6
GS_nf: 32
GS_nlayers: 6
GT_nf: 32
GT_nlayers: 6
Sanglejitter: False
Straining_num: 2560
Tanglejitter: False
Ttraining_num: 800
batchsize: 4
glyph_preserve: False
gpu: True
load_GB_name: ../save/GB-iccv.ckpt
load_GS_name: ../save/sample_v3-GS.ckpt
save_name: sample_v3-styleloss
save_path: ../save
scale_num: 4
step1_epochs: 30
step2_epochs: 40
step3_epochs: 80
step4_epochs: 10
style_loss: True
style_name: ../data/style/sample_v3.png
subimg_size: 256
text_datasize: 708
text_path: ../data/rawtext/yaheiB/train
texture_step1_epochs: 40
texture_step2_epochs: 10
--- create model ---
Traceback (most recent call last):
  File "trainTextureTransfer.py", line 74, in <module>
    main()
  File "trainTextureTransfer.py", line 37, in main
    style_targets = get_GRAM(opts.style_name, VGGfeatures, opts.batchsize, opts.gpu)
  File "/content/ShapeMatchingGAN/src/vgg.py", line 71, in get_G

Now run a test using your custom model. make sure to reference your new GS and GT models.

In [0]:
!python test.py --text_name ../data/rawtext/yaheiB/val/0801.png --scale -1 --scale_step 0.005 --structure_model ../save/sample_v3-GS.ckpt --texture_model ../save/sample_v3-GT.ckpt --result_dir ../output --name sample_v3-0801 --gpu

--- load options ---
gpu: True
name: sample_v3-0801
result_dir: ../output
scale: -1.0
scale_step: 0.005
structure_model: ../save/sample_v3-GS.ckpt
text_name: ../data/rawtext/yaheiB/val/0801.png
text_type: 0
texture_model: ../save/sample_v3-GT.ckpt
--- load data ---
--- load model ---
--- testing ---
--- save ---


In [0]:
cd ShapeMatchingGAN/src

/content/ShapeMatchingGAN/src


In [0]:
!zip sample_v3.zip /content/ShapeMatchingGAN/output/sample_v3-0801_*.png

  adding: content/ShapeMatchingGAN/output/sample_v3-0801_0.png (deflated 22%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_100.png (deflated 12%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_101.png (deflated 12%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_102.png (deflated 11%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_103.png (deflated 12%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_104.png (deflated 12%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_105.png (deflated 12%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_106.png (deflated 11%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_107.png (deflated 10%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_108.png (deflated 11%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_109.png (deflated 11%)
  adding: content/ShapeMatchingGAN/output/sample_v3-0801_10.png (deflated 21%)
  adding: content/ShapeMatchingGAN/output/s